In [1]:
##

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1678456524674_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import boto3
import pandas as pd
import numpy as np
import time
from PIL import Image
from io import BytesIO
from io import StringIO
from pyspark.sql.functions import input_file_name, split
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.image import ImageSchema
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors, VectorUDT
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Accès aux ressources du AWS S3 bucket
s3 = boto3.resource('s3')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Affichage des buckets
for bucket in s3.buckets.all():
    print(bucket.name)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

aws-logs-372821132679-eu-west-3
p8data-mato7777

In [5]:
# Affichage des n premiers fichiers du bucket p8data-mato7777
fruits_bucket = s3.Bucket('p8data-mato7777')
for file in fruits_bucket.objects.limit(3):
    label = file.key.split('/')[-2]
    print(label, file.key)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Apple_Braeburn Test/Apple_Braeburn/321_100.jpg
Apple_Braeburn Test/Apple_Braeburn/322_100.jpg
Apple_Braeburn Test/Apple_Braeburn/323_100.jpg

In [6]:
# Modèle pour classification, dernières couches tronquées
conv_base = VGG16(
    include_top=False,
    weights=None,
    pooling='max',
    input_shape=(100, 100, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
conv_base.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0     

In [8]:
# get the n first cnn_features
start = time.perf_counter()
list_path_img = []
for file in fruits_bucket.objects.limit(10000):
    obj = fruits_bucket.Object(file.key)
    label = file.key.split('/')[-2]
    response = obj.get()
    file_stream = response['Body']
    img = Image.open(file_stream)
    # convert image to flatten array
    flat_array = np.array(img).ravel().tolist()
    tensor = np.array(flat_array).reshape(1, 100, 100, 3).astype(np.uint8)
    # preprocess input
    prep_tensor = preprocess_input(tensor)
    features = conv_base.predict(prep_tensor).ravel().tolist()
    # Store file key and features
    list_path_img.append((file.key, label, features))
stop = time.perf_counter()
print(f'process, elapsed time: {stop - start:0.2f}s')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

process, elapsed time: 1985.09s

In [9]:
# Création d'un spark dataframe depuis la liste de tuples précédente
df_img = spark.createDataFrame(list_path_img, ['origin', 'label', 'cnn_features'])
# Affichage du spark dataframe
df_img.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+
|              origin|         label|        cnn_features|
+--------------------+--------------+--------------------+
|Test/Apple_Braebu...|Apple_Braeburn|[0.22748878598213...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.23494637012481...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.22700527310371...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.24659475684165...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.25048434734344...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.24582087993621...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.24824020266532...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.25295341014862...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.25764816999435...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.26455134153366...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.26553359627723...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.25584769248962...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.26627930998802...|
|Test/Apple_Braebu...|Apple_Braeburn|[0.25640931725502..

In [10]:
df_img.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- origin: string (nullable = true)
 |-- label: string (nullable = true)
 |-- cnn_features: array (nullable = true)
 |    |-- element: double (containsNull = true)

In [11]:
# Arrays en vecteurs pour la PCA
array_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_img = df_img.withColumn('cnn_vectors', array_to_vector_udf('cnn_features'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_img = df_img.select('origin', 'label', 'cnn_vectors')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Réduction PCA - k=20 choisi arbitrairement
start = time.perf_counter()
pca = PCA(k=20, inputCol='cnn_vectors', outputCol='pca_vectors')
model = pca.fit(df_img)
stop = time.perf_counter()
print(f'pca - 20 pcs, elapsed time: {stop - start:0.2f}s')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pca - 20 pcs, elapsed time: 11.04s

In [14]:
# Application PCA
df_img = model.transform(df_img)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Vecteur en Arrays
vector_to_array_udf = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_img = df_img.withColumn('arrays', vector_to_array_udf('pca_vectors'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_img = df_img.select('origin', 'label', 'arrays')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_img.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+
|              origin|         label|              arrays|
+--------------------+--------------+--------------------+
|Test/Apple_Braebu...|Apple_Braeburn|[-6.8931746, -0.1...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.9110427, -0.1...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.911124, -0.15...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.924674, -0.13...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.929066, -0.12...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.9227586, -0.1...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.9469285, -0.1...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.8489323, 0.04...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.866737, 0.046...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.8688035, 0.06...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.887347, 0.051...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.8982224, 0.06...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.904811, 0.078...|
|Test/Apple_Braebu...|Apple_Braeburn|[-6.9258437, 0.08..

In [19]:
# Passage du spark dataframe en pandas dataframe
results_df = df_img.toPandas()
results_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                               origin  ...                                             arrays
0     Test/Apple_Braeburn/321_100.jpg  ...  [-6.893174648284912, -0.1839231252670288, -0.6...
1     Test/Apple_Braeburn/322_100.jpg  ...  [-6.9110426902771, -0.1696942150592804, -0.655...
2     Test/Apple_Braeburn/323_100.jpg  ...  [-6.911124229431152, -0.15038786828517914, -0....
3     Test/Apple_Braeburn/324_100.jpg  ...  [-6.924674034118652, -0.13461808860301971, -0....
4     Test/Apple_Braeburn/325_100.jpg  ...  [-6.929066181182861, -0.12748154997825623, -0....
...                               ...  ...                                                ...
9995            Test/Lemon/73_100.jpg  ...  [-6.792480945587158, -0.8613798022270203, -0.0...
9996            Test/Lemon/74_100.jpg  ...  [-6.711960315704346, -0.8299753665924072, -0.0...
9997            Test/Lemon/75_100.jpg  ...  [-6.7024245262146, -0.830775797367096, -0.0443...
9998            Test/Lemon/76_100.jpg  ...  [-6.709100246429

In [20]:
# Sauvegarde dans le bucket s3, avec boto3
# buffer
csv_buffer = StringIO()
results_df.to_csv(csv_buffer)
# boto client
client = boto3.client('s3')
# put the object
response = client.put_object(
    Body=csv_buffer.getvalue(),
    Bucket='p8data-mato7777',
    Key='results.csv')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Résumé de l'upload
response

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '8H2051S4G5884RF6', 'HostId': 'NIkxTVoX6se+aEhqt042MxORHMilEiryE3wCrby+7rLBFxvSrPtFuXtDGsz6oagvO1/pQIVUUqA=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'NIkxTVoX6se+aEhqt042MxORHMilEiryE3wCrby+7rLBFxvSrPtFuXtDGsz6oagvO1/pQIVUUqA=', 'x-amz-request-id': '8H2051S4G5884RF6', 'date': 'Fri, 10 Mar 2023 14:33:40 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"444a019e8d51fd58326e7207200eff76"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'ETag': '"444a019e8d51fd58326e7207200eff76"', 'ServerSideEncryption': 'AES256'}